# Classification of anomalous timeseries

In Stackstate we are challenged with a number of problems from IT operations that we are solving using Machine Learning techniques.

One of the problems that is very important is classification of timeseries for further automatic problem detection at scale.
Usually the metrics are coming from different sources and monitoring tools. The metrics typically are ["Golden signals"](https://landing.google.com/sre/sre-book/chapters/monitoring-distributed-systems/#xref_monitoring_golden-signals), resource utilization (CPU/RAM/Disk), business KPIs (Latency).

In this notebook I'd like to give an introduction of how classification of time series can be performed. In order to do that I formulate the following questions.

## Research Questions

1. How does anomaly look like and how often it occurs? 
2. Can anomaly be detected using a classifier?
3. How to overcome problems with training for anomaly detection.


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
# Including parent path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import os
import sys
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta as td

from sklearn.model_selection import train_test_split

import unittest
import numpy as np
import pandas as pd

from preprocessing.test_common import make_labels, make_series
from preprocessing.helper import TimeseriesHelper
from preprocessing.preprocessing import TimeSeriesPreprocessor
from preprocessing.feature_engineering import TimeSeriesFeatureEngineering
from models.xgboost import XGBoostModel

from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()
warnings.filterwarnings('ignore')
print(sys.path)

['/anaconda3/envs/time-series/lib/python36.zip', '/anaconda3/envs/time-series/lib/python3.6', '/anaconda3/envs/time-series/lib/python3.6/lib-dynload', '', '/anaconda3/envs/time-series/lib/python3.6/site-packages', '/anaconda3/envs/time-series/lib/python3.6/site-packages/IPython/extensions', '/Users/konst/.ipython', '/Users/konst/stackvista/time-series']


## 1. Loading the data

I have chosen the NAB dataset for anomaly research because it is publicly available dataset that contains a representative set of timeseries with labeled anomalies.

It is very important that the anomalies are labeled by the experts as they provide necessary context and establish necessary casual relationship between change in metric - a signal and corresponding issue - symptom. 

If the time series is not labled then it is difficult to conclude if the change in the signal that reached a threshold or a frequency change indicating typical changepoint or actual problem.

In [4]:
helper = TimeseriesHelper()

metric_map = helper.load_multiple_series(
    ["realAWSCloudwatch/ec2_cpu_utilization_5f5533.csv", 
     "realAWSCloudwatch/ec2_cpu_utilization_53ea38.csv",
     "realAWSCloudwatch/ec2_disk_write_bytes_c0d644.csv",
     "realAWSCloudwatch/grok_asg_anomaly.csv",
     "realTweets/Twitter_volume_AMZN.csv",
     "realTweets/Twitter_volume_UPS.csv",
     "realTraffic/TravelTime_387.csv",
     "realKnownCause/ec2_request_latency_system_failure.csv",
     "realKnownCause/machine_temperature_system_failure.csv",
     "realKnownCause/ambient_temperature_system_failure.csv"])

FileNotFoundError: File b'data/NAB/realAWSCloudwatch/ec2_cpu_utilization_5f5533.csv' does not exist

## 2. How does anomaly look like and how often it occurs?

Let's now examine the number of NAB metrics we preselected, plot them with corresponding labels and give the definition of anomaly according to what the experts thought the anomaly was.

In [ ]:
helper.plot_metrics(metric_map, figsize=(18, 50))

## Anomaly Definition

From the charts above it is clear that the anomaly is:
- Sudden metric value spike or collapse
- Sudden change in variance
- Sudden level shift
- Any combination of one of the mentioned above

Anomalies are quite rare. I foresee issues due to class disbalance where it is easier for a model to always say "No Anomaly" and still get 99.99% accuracy.

Note that we are analyzing the anomalies pretty much without the context, observing the metric data itself. 


### Data Preprocessing

Before extracting features from time series it is always a good practice to perform data cleaning and preprocessing.

I will execute the following preprocessing steps:
1. Cleaning from missing values
2. Aggregating into 5 minute buckets.
3. If after aggregation there are missing values they are imputed using backfill and forwardfill methods.

The result dataset is ready for feature engineering. 

In [ ]:
ec2_request_latency_system_failure_file = "realKnownCause/ec2_request_latency_system_failure.csv" # bad
ec2_cpu_utilization_failure_file = "realAWSCloudwatch/ec2_cpu_utilization_5f5533.csv" 
Twitter_volume_AMZN_file = "realTweets/Twitter_volume_AMZN.csv" # good
machine_temperature_system_failure_file = "realKnownCause/machine_temperature_system_failure.csv" # good

df1 = metric_map[ec2_request_latency_system_failure_file]
df2 = metric_map[ec2_cpu_utilization_failure_file]
df3 = metric_map[Twitter_volume_AMZN_file]
df4 = metric_map[machine_temperature_system_failure_file]

train = df1

In [ ]:
from sklearn.preprocessing import MinMaxScaler

input_variables = ['y']
output_variable = 'label'

preprocessor = TimeSeriesPreprocessor(
    window_size_seconds = 7200 * 3,
    window_shift = 300,
    horizon_shift_seconds = 3600,
    probe_period_seconds = 300,
    scaler = MinMaxScaler())

series = preprocessor.prepare_series(train,
        input_vars = input_variables, output_vars = [output_variable],
        numeric_vars = input_variables, auto_impute = input_variables)


## 3. Automatic Feature Engineering

Feature extraction and engineering is laborious process that takes considerable amount of time therefore I'd like to perform it automatically. 

For this purpose I will use [tsfresh](https://tsfresh.readthedocs.io/en/latest/) library that does automatic feature extraction of around [700 features](https://tsfresh.readthedocs.io/en/latest/text/list_of_features.html) and then execute statistical tests to select only those features that are relevant for classification task. For more details on how it does that I am forwarding the reader to the documentation on [feature extraction](https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html) and [feature filtering](https://tsfresh.readthedocs.io/en/latest/text/feature_filtering.html)

We extract and select the number of features from rolling window of 20 points which corresponds to ~ 100 minutes of time.

In [ ]:
dataset = None

In [ ]:
feature_engineering = TimeSeriesFeatureEngineering(
      x_columns = input_variables,
      roll_shift = 20,
      ts_variable = 'timestamp',
      y_column = output_variable)

In [ ]:
dataset = feature_engineering.make_features(series)

Since feature engineering is quite CPU and RAM intensive process I dont want to repeat it often during experimenting and I am happy to reuse this result and therefore save it to a file.

In [ ]:
if dataset is not None:
    dataset.to_csv("./extracted_features.csv")
else:
    dataset = pd.read_csv("./df4_features.csv")

## 4. Model training

I am splitting the dataset to train/validation/test sets as always in such cases:


In [ ]:
train_test_split_ratio = 0.2
train_valid_split_ratio = 0.2
seed = 42

input_features = list(dataset.columns)
input_features.remove(output_variable)

X_train, X_test, y_train, y_test = train_test_split(dataset[input_features], dataset[output_variable], test_size=train_test_split_ratio, random_state = seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = train_valid_split_ratio, random_state = seed)


### 4.1 Training Gradient Boosting tree model

I am going to train Gradient Boosting Tree as a classifier. This type of model has very good performance although it is also prone to overfitting.

In [ ]:
model = XGBoostModel()

model.fit(X_train.as_matrix(), y_train.as_matrix(), X_valid.as_matrix(), y_valid.as_matrix())

Y_train_pred = model.predict(X_train.as_matrix())
helper.print_results("Training", helper.evaluate(y_train, Y_train_pred))

Y_valid_pred = model.predict(X_valid.as_matrix())
helper.print_results("Validation", helper.evaluate(y_valid, Y_valid_pred))

y_test_pred = model.predict(X_test.as_matrix())
helper.print_results("Test", helper.evaluate(y_test, y_test_pred))


Not bad considering the fact that there is only ~1 % positive samples 

The result overview over all 4 series is given below:

| F1 scores                   | training | validation | test |
|-----------------------------|----------|------------|------|
| request latency failure     | 0.98     | 1.0        | 1.0  |
| cpu utilization failure     | 0.68     | 0.62       | 0.47 |
| twitter volume AMZN         | 0.83     | 0.74       | 0.61 |
| Machine temperature failure | 0.60     | 0.60       | 0.5  |

It seems that in certain cases the classifier a bit overfits the data.
It is also noticible that performance is not great on some datasets and from confusion matrix it looks like classifier tends to infer "not a anomaly" quite often.

### 4.2 Class Disbalance

As it was expected the anomalies are quite rare therefore one of the big challenges in machine learning on anomalous time series is how to train decent classifier with only a couple anomalous samples.

In order to fix the class disblance I am going to artificially increase the number of anomalous samples by using [Synthetic Minority Over-sampling Technique](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html). This will allow to balance the number of positive and negative samples and I avoid cheap wins like always detect "Not anomaly".

The code and results of those experiments are given below:

In [ ]:
print("class distribution in original training data.\n", y_train.groupby(by=lambda v: y_train.loc[v]).count())
X_upsampled, y_upsampled = feature_engineering.class_imbalance_fix(X_train, y_train)
elements, counts_elements = np.unique(y_upsampled, return_counts=True)
print("class distribution in upsampled training data.")
print(elements)
print(counts_elements)

model = XGBoostModel()
model.fit(X_upsampled, y_upsampled, X_valid.as_matrix(), y_valid.as_matrix())

Y_train_pred = model.predict(X_train.as_matrix())
helper.print_results("Training", helper.evaluate(y_train, Y_train_pred))

Y_valid_pred = model.predict(X_valid.as_matrix())
helper.print_results("Validation", helper.evaluate(y_valid, Y_valid_pred))

y_test_pred = model.predict(X_test.as_matrix())
helper.print_results("Test", helper.evaluate(y_test, y_test_pred))


It looks like with the increasing of the number of positive samples using SMOTE our model perform much better.

| F1 scores                   | training | validation | test |
|-----------------------------|----------|------------|------|
| request latency failure     | 0.98     | 1.0        | 1.0  |
| cpu utilization failure     | 0.94     | 0.66       | 0.8  |
| twitter volume AMZN         | 0.61     | 0.39       | 0.59 |
| Machine temperature failure | 0.60     | 0.60       | 0.5  |


### 4.3 Futher increase of accuracy with the window size increase

It is possible to increase accuracy further by using bigger windows. For example: 